In [ ]:
# Imports
import pyTigerGraph as tg
import json
import os

# Connection parameters
hostName = "https://c2fa4efab8ca4cb28b9d08be859caa2c.i.tgcloud.io/"
secret = "fdbu0naud8tu2ik5imlb758s96m2jssq"

conn = tg.TigerGraphConnection(host=hostName, gsqlSecret=secret, graphname="MyGraph")
conn.getToken(secret)

### Define and Publish Graph Schema

Nodes = ETH Users/wallets: ID, label <br />
Directed Edges = Transaction flow: From_ID, To_ID, ETH_amount, timestamp

In [ ]:
# DEFINE / CREATE ALL EDGES AND VERTICES in Global View
results = conn.gsql('''
  USE GLOBAL
  CREATE VERTEX Wallet (PRIMARY_ID id INT, label FLOAT) WITH primary_id_as_attribute="true"
  CREATE DIRECTED EDGE sent_eth (from Wallet, to Wallet, amount FLOAT, sent_date INT) WITH REVERSE_EDGE="reverse_sent_eth"
  CREATE DIRECTED EDGE received_eth (from Wallet, to Wallet, amount FLOAT, receive_date INT) WITH REVERSE_EDGE="reverse_received_eth"
''')
print(results)

### Create the Ethereum Transaction Graph

In [ ]:
# Create a new graph from the global schema
results = conn.gsql('''
  CREATE GRAPH MyGraph2(Wallet, sent_eth, reverse_sent_eth, received_eth, reverse_received_eth)
''')
print(results)

In [ ]:
# connect to the newly created graph
conn.graphname="MyGraph2"
secret = conn.createSecret()
conn = tg.TigerGraphConnection(host=hostName, gsqlSecret=secret, graphname="MyGraph2")
conn.getToken(secret)
conn.getSchema()

### Create Loading Jobs

#### Wallets

In [ ]:
# Custom loading job that maps the values of nodes.csv to VERTEX attributes
results = conn.gsql('''
  USE GRAPH MyGraph2
  BEGIN
  CREATE LOADING JOB load_wallets FOR GRAPH MyGraph2 {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Wallet VALUES($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')
print(results)

#### Transactions

In [ ]:
# Custom loading job that maps the values of edges.csv to EDGE attributes
results = conn.gsql('''
  USE GRAPH MyGraph2
  BEGIN
  CREATE LOADING JOB load_transactions FOR GRAPH MyGraph2 {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE sent_eth VALUES($1, $0, $2, $3) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  LOAD MyDataSource TO EDGE received_eth VALUES($0, $1, $2, $3) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  }
  END''')
print(results)

### Load Data

In [ ]:
# Load the nodes file with the 'load_wallets' job
nodes_file = 'data/nodes.csv'
results = conn.runLoadingJobWithFile(filePath=nodes_file, fileTag='MyDataSource', jobName='load_wallets')
print(json.dumps(results, indent=2))

In [ ]:
# Load the edges file with the 'load_transactions' job
edges_file = 'data/edges.csv'
results = conn.runLoadingJobWithFile(filePath=edges_file, fileTag='MyDataSource', jobName='load_transactions')
print(json.dumps(results, indent=2))

### Exploring the Graph

In [ ]:
# TODO - ETHAN

# write GSQL queries to get some summary statistics on edges/nodes
# use this as reference: https://colab.research.google.com/drive/1JhYcnGVWT51KswcXZzyPzKqCoPP5htcC?usp=sharing#scrollTo=Zerob6pDgmq2 